### Домашняя работа № 2

**Цель работы:** научиться использовать Python для загрузки данных
из интернет с помощью парсинга Web-страниц и через REST API.

В этой работе мы будем загружать данные о вакансиях
разработчиков Python с сайта hh.ru.

**Задание**
Разработайте программу на Python, которая скачивает
информацию о вакансиях разработчиков Python среднего уровня
(middle python developer).

Программа должна получать следующую информацию о каждой
вакансии:
* Название компании.
* Название позиции (должности).
* Описание вакансии.
* Список ключевых навыков.

Программа должна получать данные о вакансиях двумя
способами:
1. Парсинг Web-страниц сайта hh.ru.
2. Вызов API hh.ru (https://dev.hh.ru/).

Программа должна скачать информацию о 100 вакансиях.

Скачанные данные о вакансиях запишите в базу данных SQLite, таблица
vacancies, столбцы:
* company_name (название компании).
* position (название позиции).
* job_description (описание вакансии).
* key_skills (список ключевых навыков).

**Задание повышенной сложности:** программа должна загружать данные о
вакансиях асинхронно.

Для сдачи задания пришлите исходные коды программ на Python или
ссылку на git репозиторий с исходным кодом.

#### Атрибуты для парсинга
##### Основные:
|Атрибут|Описание|Название атрибута в источнике|
|:------|:-------|:----------------------------|
|**company_name**|название компании|employer {name, url, alternate_url}|
|**position**|название позиции|name|
|**job_description**|описание вакансии|snippet, description|
|**key_skills**|список ключевых навыков|key_skills [{name}]|
##### Дополнительные:
|Атрибут|Описание|Название атрибута в источнике|
|:------|:-------|:----------------------------|
|area|территория|area {name}|
|url|основаня ссылка на вакансию
|alternate_url|альтернативная ссылка на вакансию|
|published_at|дата публикации вакансии|
|created_at|дата создания вакансии|
|archived|признак архивности вакансии|
|salary|зарплатная вилка|salary {from, to, currency, gross}|
|responsibility|обязанности|
|experience|опыт работы|experience {name}|
|employment|занятость|employment {name}|
|accredited_it_employer|имеет ли компания аккредитацию IT-компании|
|professional_roles|профессиональная область|professional_roles {name}|

### Решение

In [1]:
import pandas as pd
# from typing import List
# import requests
from time import sleep
import sqlite3
# from pathlib import Path
import json5
from utils import (areas_parser, get_query, execute, get_data_by_api, get_table,
                   get_view, data_parser, list_to_str, update_table, normalizer)

In [9]:
settings = json5.load(open('settings.json', encoding='utf-8'))

# globals().update(**settings)
db_name = settings['db_name']
url_areas = settings['url_areas']
url_vac = settings['url_vac']
country = settings['country']
regions = settings['regions']
url_params = settings['url_params']

areas_dct = areas_parser(url_areas, country, regions)  # получаем id заданных регионов
print(f'id регионов: {areas_dct}')
areas_lst = list(areas_dct)

url_params['area'] = areas_lst

id регионов: {1: 'Москва', 2: 'Санкт-Петербург', 1438: 'Краснодарский край'}


In [12]:
def get_vacancies(url, params, vacancies=None):
    """Получает список вакансий по заданному URL с параметрами"""
    if vacancies is None:
        vacancies = []

    if params.get('page', 0) < params.get('pages', 1):
        data = get_data_by_api(url, params)
        _vacancies = data.get('items', [])
        found, page, pages = data.get('found', 0), data.get('page', 0), data.get('pages', 1)
        params.update(found=found, page=page + 1, pages=pages)
        print(f'\tfound: {found}, pages: {pages}, page: {page} in get_vac')
        vacancies += _vacancies

        print('\tpage after in get_vac', params.get('page'), page)

        # Количество вакансий
        print('Количество спарсенных со стр', page,
              'вакансий по заданным параметрам:', len(_vacancies),
              'всего спарсено вакансий:', len(vacancies), end='\n\n')

        if len(vacancies) < (found if found < 100 else 100) and page < pages:
            sleep(3)
            get_vacancies(url, params, vacancies)
            page += 1
            print('\tpage after recursion in get_vac', params.get('page'), page)

    print('\tpage final in get_vac', params.get('page'))
    return vacancies


def attributes_processing(vacancies_df: pd.DataFrame):
    """Обработка DF вакансий"""
    # Парсим атрибуты, достаём названия у тех, которые имеют форму словарей/списков
    # и добавляем в DF вакансий
    print()
    for attrib in vacancies_df:
        if attrib in ('employer', 'salary'):
            continue
        print('\tParsing attribute', attrib)
        data = vacancies_df[attrib].apply(data_parser)
        if isinstance(data, pd.DataFrame):
            if 'name' in data:
                vacancies_df[attrib] = data['name']
        else:
            vacancies_df[attrib] = data
    print()


def employers_proccessing(vacancies_df: pd.DataFrame):
    # Парсим работодателей в DF для создания отдельной таблицы
    employers_df = vacancies_df.employer.apply(data_parser)
    # Убираем лишний атрибут logo_urls
    employers_df.drop(columns='logo_urls', inplace=True)

    vacancies_df[
        [
            'company_id',
            'company_name',
            'company_trusted'
        ]
    ] = employers_df[
        [
            'id',
            'name',
            'trusted'
        ]
    ]

#     vacancies_df = pd.concat([vacancies_df,
#                               employers_df.add_prefix('company_')],
#                              axis=1)
    # Убираем дубли
    employers_df = employers_df[~employers_df.duplicated()]
    # Запись DF в таблицу БД employers
    update_table('employers', db_name, employers_df)


def key_skills_processing(vacancies_df: pd.DataFrame):
    # Сздаём отдельный DF с ключевыми скиллами
    key_skills_df = vacancies_df[
        ['id', 'key_skills']
    ].rename(
        columns=dict(
            id='vacancy_id',
            key_skills='name'
        )
    )
    # Разбиваем ключевые скиллы, записанные через запятую на списки
    key_skills_df['name'] = key_skills_df.name.apply(lambda x: x.split(',')
                                                     if isinstance(x, str)
                                                     else x)
    # Растягиваем списки на атомарные значения после разбивки
    key_skills_df = key_skills_df.explode('name', ignore_index=True)
    # Нормализуем ключевые слова
    key_skills_df['normalized_name'] = key_skills_df.name.apply(normalizer)

    # Убираем дубли
    key_skills_df = key_skills_df[~key_skills_df.duplicated()]
    # Запись DF в таблицу БД key_skills
    update_table('key_skills', db_name, key_skills_df)


def vacancies_processing(df: pd.DataFrame = pd.DataFrame()):
    """Обработка списка вакансий и преобразование в таблицы"""
    print('page before in proc', url_params.get('page'))
    fill_vac = [{}] * len(df)
    vacancies = get_vacancies(url_vac, url_params, fill_vac)
    found, page, pages = (url_params.get('found', 0),
                          url_params.get('page', 0),
                          url_params.get('pages', 0))
    print('page after in proc', page)
    print('num vac =', len(vacancies))
    print('len df is:', len(df))

    # Парсим список вакансий в DF и оставляем только нужные атрибуты
    vacancy_attribs = ['id', 'name', 'url', 'alternate_url', 'employer', 'area',
                       'employment', 'salary', 'experience', 'professional_roles',
                       'published_at', 'created_at', 'archived']
    global vacancies_df
    vacancies_df = pd.DataFrame(vacancies)[vacancy_attribs]
    # очистка DF от добавленных пустых строк
    vacancies.clear()  # очистка массива со спарсенными ранее вакансиями
    vacancies_df = vacancies_df.dropna(how='all').reset_index(drop=True)
    vacancies_df.rename(columns=dict(name='position'), inplace=True)
    # Убираем архивные вакансии
    vacancies_df = vacancies_df[~vacancies_df.archived.fillna(False)]

    # Обрабатываем атрибуты
    attributes_processing(vacancies_df)

    # Записываем спарсенные компании в таблицу employers
    employers_proccessing(vacancies_df)

    # Парсим зарплаты
    salary_df = vacancies_df.salary.apply(data_parser)
    vacancies_df = pd.concat([vacancies_df,
                              salary_df.add_prefix('salary_')], axis=1)

    # Парсим дополнительные атрибуты по каждой вакансии по API и записываем в DF вакансий
    vacancies_df['details'] = vacancies_df.url.apply(get_data_by_api)

    details_df = vacancies_df.details.apply(data_parser)
    details_df = details_df[
        ['description',
         'key_skills']
    ].rename(
        columns=dict(
            description='job_description'
        )
    )

    vacancies_df = pd.concat([vacancies_df, details_df], axis=1)

    # Обработка атрибута key_skills
    vacancies_df['key_skills'] = vacancies_df.key_skills.apply(list_to_str)
    # Оставляем только те вакансии, в которых указаны ключевые навыки,
    # которые разместили проверенные работодатели,имеющие аккредитацию IT компании
    vacancies_df = vacancies_df[vacancies_df.company_trusted &
                                # vacancies_df.company_accredited_it_employer &
                                vacancies_df.key_skills.notna()]
    # убираем лишние атрибуты
    vacancies_attribs = [
        'id',
        'position',
        'job_description',
        'url',
        'alternate_url',
        'area',
        'employment',
        'experience',
        'professional_roles',
        'key_skills',
        'salary_from',
        'salary_to',
        'salary_currency',
        'salary_gross',
        'company_id',
        'company_name',
        'published_at',
        'created_at',
        'archived'
    ]
    vacancies_df = pd.concat([df, vacancies_df[vacancies_attribs]], ignore_index=True)
    # Убираем дубли
    vacancies_df = vacancies_df[~vacancies_df.id.duplicated()]
    print('len vacancies_df after filters is:', len(vacancies_df))
    if len(vacancies_df) < (found if found < 100 else 100) and page < pages:
        # Если количество вакансий после фильтров оказалось меньше 100,
        # и при этом найдено более 100 вакансий, то запускаем процесс заново
        sleep(10)  # ожидание чтобы не попасть на капчу
        vacancies_processing(vacancies_df.copy(deep=True))
        print('len vacancies_df after recursion is:', len(vacancies_df))
        return
    print('len vacancies_df final is:', len(vacancies_df))

    update_table('vacancies', db_name, vacancies_df)

    # Записываем спарсенные клчевые навыки в таблицу key_skills
    key_skills_processing(vacancies_df)

    # Очистка БД от мусора
    execute('vacuum', db_name)


if __name__ == '__main__':
    # Создаём таблицу employers в БД
    execute(get_query('create_employers.sql'), db_name)
    # Создаём таблицу vacancies в БД
    execute(get_query('create_vacancies.sql'), db_name)
    # Создаём таблицу key_skills в БД
    execute(get_query('create_key_skills.sql'), db_name)

    vacancies_processing()

page before in proc 0
URL: https://api.hh.ru/vacancies?text=middle+python&search_field=name&page=0&per_page=50&archived=False&area=1&area=2&area=1438
GET request sucessful
	found: 130, pages: 3, page: 0 in get_vac
	page after in get_vac 1 0
Количество спарсенных со стр 0 вакансий по заданным параметрам: 50 всего спарсено вакансий: 50

URL: https://api.hh.ru/vacancies?text=middle+python&search_field=name&page=1&per_page=50&archived=False&area=1&area=2&area=1438&found=130&pages=3
GET request sucessful
	found: 130, pages: 3, page: 1 in get_vac
	page after in get_vac 2 1
Количество спарсенных со стр 1 вакансий по заданным параметрам: 50 всего спарсено вакансий: 100

	page final in get_vac 2
	page after recursion in get_vac 2 1
	page final in get_vac 2
page after in proc 2
num vac = 100
len df is: 0

	Parsing attribute id
	Parsing attribute position
	Parsing attribute url
	Parsing attribute alternate_url
	Parsing attribute area
	Parsing attribute employment
	Parsing attribute experience
	Pa

### Проверка результата

In [14]:
# Проверяем получившийся результат по компаниям
employers_df = get_table('employers', db_name)
employers_df

,id,name,url,alternate_url,vacancies_url,accredited_it_employer,trusted
0,5843588.0,Webtronics,https://api.hh.ru/employers/5843588,https://hh.ru/employer/5843588,https://api.hh.ru/vacancies?employer_id=5843588,0.0,1
1,4250943.0,hotellab.io,https://api.hh.ru/employers/4250943,https://hh.ru/employer/4250943,https://api.hh.ru/vacancies?employer_id=4250943,0.0,1
2,23314.0,НПФ ФИТО,https://api.hh.ru/employers/23314,https://hh.ru/employer/23314,https://api.hh.ru/vacancies?employer_id=23314,0.0,1
3,4960417.0,Талала,https://api.hh.ru/employers/4960417,https://hh.ru/employer/4960417,https://api.hh.ru/vacancies?employer_id=4960417,0.0,1
4,3934385.0,JetLend,https://api.hh.ru/employers/3934385,https://hh.ru/employer/3934385,https://api.hh.ru/vacancies?employer_id=3934385,0.0,1
...,...,...,...,...,...,...,...
100,1102601.0,Группа Самолет,https://api.hh.ru/employers/1102601,https://hh.ru/employer/1102601,https://api.hh.ru/vacancies?employer_id=1102601,0.0,1
101,3785152.0,Eqvanta,https://api.hh.ru/employers/3785152,https://hh.ru/employer/3785152,https://api.hh.ru/vacancies?employer_id=3785152,0.0,1
102,1144566.0,ЛАЙФСТРИМ,https://api.hh.ru/employers/1144566,https://hh.ru/employer/1144566,https://api.hh.ru/vacancies?employer_id=1144566,1.0,1
103,976931.0,The Skolkovo Institute of Science and Technology,https://api.hh.ru/employers/976931,https://hh.ru/employer/976931,https://api.hh.ru/vacancies?employer_id=976931,0.0,1


In [15]:
# Проверяем получившийся результат по вакансиям
get_table('vacancies', db_name)

,id,position,job_description,url,alternate_url,area,employment,experience,professional_roles,key_skills,salary_from,salary_to,salary_currency,salary_gross,company_id,company_name,published_at,created_at,archived
0,55088580,"Разработчик Python (Middle, Senior)",<p>NGENIX — лидер на рынке облачных решений и ...,https://api.hh.ru/vacancies/55088580?host=hh.ru,https://hh.ru/vacancy/55088580,Москва,Полная занятость,От 3 до 6 лет,"Программист, разработчик","Python, PostgreSQL, SQL, MySQL, Scrum",NaN,NaN,None,NaN,656166.0,NGENIX,2023-07-26T09:57:45+0300,2023-07-26T09:57:45+0300,0
1,67115536,Middle Python разработчик (Django),<p><em>МедРокет</em> — платформа IT-решений дл...,https://api.hh.ru/vacancies/67115536?host=hh.ru,https://hh.ru/vacancy/67115536,Краснодар,Полная занятость,От 1 года до 3 лет,"Программист, разработчик","Python, Django Framework",160000.0,240000.0,RUR,1.0,1545815.0,МедРокет,2023-07-24T08:28:25+0300,2023-07-24T08:28:25+0300,0
2,71226010,Middle Python backend developer (ML Space AI C...,<p>Приглашаем <strong>Middle Backend Developer...,https://api.hh.ru/vacancies/71226010?host=hh.ru,https://hh.ru/vacancy/71226010,Москва,Полная занятость,От 1 года до 3 лет,"Программист, разработчик","Backend, Middle, Kubernetes, Python, Linux, Go...",NaN,NaN,None,NaN,3853446.0,Cloud.ru,2023-06-30T22:38:52+0300,2023-06-30T22:38:52+0300,0
3,71226025,Middle/Senior Python Developer (Platform Virtu...,<p><strong>Вам предстоит:</strong></p> <ul> <l...,https://api.hh.ru/vacancies/71226025?host=hh.ru,https://hh.ru/vacancy/71226025,Москва,Полная занятость,От 3 до 6 лет,"Программист, разработчик","Python, Linux, SQL, ООП, Английский язык, Kube...",NaN,NaN,None,NaN,3853446.0,Cloud.ru,2023-06-30T22:38:29+0300,2023-06-30T22:38:29+0300,0
4,72687504,Middle Python developer,<p>Мы занимаемся разработкой и поддержкой собс...,https://api.hh.ru/vacancies/72687504?host=hh.ru,https://hh.ru/vacancy/72687504,Москва,Полная занятость,От 3 до 6 лет,"Программист, разработчик","Python, Linux, MySQL, Nginx, SQL, PostgreSQL, ...",150000.0,200000.0,RUR,0.0,4920624.0,Селфсек,2023-07-27T19:41:32+0300,2023-07-27T19:41:32+0300,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,84075135,Backend Middle Developer / Python разработчик,<p><strong>Международная ĸомпания </strong><st...,https://api.hh.ru/vacancies/84075135?host=hh.ru,https://hh.ru/vacancy/84075135,Москва,Полная занятость,От 1 года до 3 лет,"Программист, разработчик","Python, Django Framework, PostgreSQL, Redis, Р...",NaN,NaN,None,NaN,4250943.0,hotellab.io,2023-07-26T15:57:41+0300,2023-07-26T15:57:41+0300,0
102,84112326,Разработчик python (Middle/Senior/Lead),<p>Мы ищем кандидатов для Микрофинансовой комп...,https://api.hh.ru/vacancies/84112326?host=hh.ru,https://hh.ru/vacancy/84112326,Москва,Полная занятость,Более 6 лет,"Программист, разработчик",Python,NaN,NaN,None,NaN,4561908.0,Центр развития Среда возможностей,2023-07-27T10:38:42+0300,2023-07-27T10:38:42+0300,0
103,84122407,Middle Python Developer,<p><strong>Кого мы ищем:</strong></p> <p>Сейча...,https://api.hh.ru/vacancies/84122407?host=hh.ru,https://hh.ru/vacancy/84122407,Санкт-Петербург,Полная занятость,От 3 до 6 лет,"Программист, разработчик","Python, Django Framework, PostgreSQL, GraphQL,...",NaN,NaN,None,NaN,1829949.0,Звук,2023-07-27T11:47:20+0300,2023-07-27T11:47:20+0300,0
104,84148906,Middle Python Developer,"<p><strong>S7 TechLab</strong> – IT компания, ...",https://api.hh.ru/vacancies/84148906?host=hh.ru,https://hh.ru/vacancy/84148906,Москва,Полная занятость,От 1 года до 3 лет,"Программист, разработчик","Python, API, docker, ci/cd",NaN,NaN,None,NaN,766468.0,Группа компаний С7,2023-07-27T14:58:02+0300,2023-07-27T14:58:02+0300,0


In [7]:
execute('vacuum', db_name)

-1

In [16]:
# Проверяем получившийся результат по ключевым навыкам
key_skills_df = get_table('key_skills', db_name)
key_skills_df

,id,vacancy_id,name,normalized_name
0,1,82841724,Git,git
1,2,82841724,Redis,redis
2,3,82841724,PostgreSQL,sql
3,4,82841724,Python,python
4,5,82841724,REST,rest api
...,...,...,...,...
795,796,83570495,asinc.io,асинхронное программирование
796,797,83570495,fast api,rest api
797,798,83570495,rest api,rest api
798,799,83570495,kafka,kafka


In [17]:
# Проверяем сколько всего вакансий, соответствующих требованиям спарсилось в БД
stmt_total_count = get_query('count_total_vacancies.sql')

get_view(stmt_total_count, db_name)

,cnt_vac
0,106


### Топ-10 самых востребованных навыков на позицию middle python developer

In [18]:
# Подсчёт методами Pandas
key_skills_df.normalized_name.value_counts().to_frame('counts').head(10)

,counts
python,109
sql,106
rest api,57
django,53
git,39
docker,36
linux,33
redis,20
rabbitmq,19
flask,18


In [19]:
# Топ-10 самых востребованных ключевых навыков на позицию middle_python (запросом)
stmt_top10 = get_query('calculate_top10_key_skills.sql')
get_view(stmt_top10, db_name)

,normalized_name,counts
0,python,109
1,sql,106
2,rest api,57
3,django,53
4,git,39
5,docker,36
6,linux,33
7,redis,20
8,rabbitmq,19
9,асинхронное программирование,18
